## Generowanie raportów pdf
Wygeneruję tutaj raporty dla każdego z ekonomistów, potem będą do pobrania ze strony www

In [1]:
#!pip install reportlab
import pandas as pd
import pickle
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.units import inch
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

In [2]:
# Czytanie ekonomistów:
data = pd.read_csv('../Polish_all_economists_24_01.csv')
# Wczytywanie pliku CrossRef.pkl

with open('../CrossRef.pkl', 'rb') as fp:
    CrossRef = pickle.load(fp)

In [5]:
pdfmetrics.registerFont(TTFont('Verdana', 'Verdana.ttf'))

for i in range(len(data)):
    id=data.loc[i, "ID"]
    id_long=f"id_{id}"
    canvas = Canvas(f"{id_long}.pdf")
    canvas.setPageSize((11.7*inch, 11.7*inch))
    canvas.setFont("Verdana", 18)
    number = i
    
    name = data.loc[number,"Autor"]
    canvas.drawString(0.6*inch, 11*inch, f"Imię i nazwisko: {name}")
    canvas.setFont("Verdana", 12)
    inst = data.loc[number,"Instytucja"]
    canvas.drawString(0.6*inch, 10*inch, f"Główna afiliacja:  {inst}")
    cit = data.loc[number,"Cytowania"]
    auto = data.loc[number,"Auto_cytowania"]
    h_in = data.loc[number,"H_index"]
    canvas.drawString(0.6*inch, 9.6*inch, f"Liczba cytowań: {cit}. Liczba autocytowań: {auto}. H-index: {h_in}")
    art = data.loc[number,"Artykuły"]
    book = data.loc[number,"Książki"]
    chap = data.loc[number,"Rozdziały"]
    pap = data.loc[number,"Papers"]
    canvas.drawString(0.6*inch, 9.2*inch, f"Liczba artykułów: {art}. Liczba książek: {book}. Rozdziały: {chap}. Papers: {pap}")
    start = data.loc[number,"rok_start"]
    koniec = data.loc[number,"rok_koniec"]
    canvas.drawString(0.6*inch, 8.8*inch, f"Rok rozpoczęcia działalności naukowej: {start}. Ostatni aktywny rok: {koniec}.")
    canvas.drawString(0.6*inch, 8.4*inch, "Wybrane artykuły lub inne prace z usługi Cross-Ref:")
    canvas.setFont("Verdana", 10)
    try:
        for i in range(len(CrossRef[id_long]["pubs"]["message"]["items"])):
            try:
                t = CrossRef[id_long]["pubs"]["message"]["items"][i]["title"][0]
            except KeyError as e:
                t = "Tytuł niedostępny"
            try:
                x = CrossRef[id_long]["pubs"]["message"]["items"][i]["DOI"]
                d = f"DOI: {x}"
            except KeyError as e:
                d = " "
            canvas.drawString(0.6*inch, (8-i*0.2)*inch, f"{t}, {d}")
    except KeyError as e:
        canvas.drawString(0.6*inch, 8*inch, "Niestety nie uzyskano danych z usługi Cross-Ref")

    try:
        ktos = list(ludzie[id]["works_with"][0:5]["Osoba"])
        ktos_id = list(ludzie[id]["works_with"][0:5]["ID"])
    except:
        ktos = ""

    canvas.setFont("Verdana", 12)
    canvas.drawString(0.6*inch, 3.6*inch, "Dominujący współpracownicy:")
    canvas.setFont("Verdana", 10)
    for i in range(len(ktos)):
        canvas.drawString(0.6*inch, (3.4-i*0.2)*inch, f"{ktos[i]}, REPEC_ID: {ktos_id[i]}")

    canvas.setFont("Verdana", 8)
    canvas.drawString(0.6*inch, 0.5*inch, "Wykonali: Tymoteusz Mętrak (t.metrak@student.uw.edu.pl) & Wojciech Szymczak (w.szymczak2@student.uw.edu.pl)")
    canvas.drawString(0.6*inch, 0.3*inch, "Stan danych na 30 grudnia 2023 r.")

    canvas.save()